# Qwen3 Embedding & Reranker - Chuyển Đổi ONNX và Quantization

**Mô tả:**
- Tải xuống models từ HuggingFace
- Chuyển đổi sang định dạng ONNX
- Quantize (INT8 Dynamic) để giảm kích thước
- Tạo Model Card và hướng dẫn upload thủ công

**Models:**
- `Qwen/Qwen3-Embedding-0.6B` - Model embedding cho semantic search
- `Qwen/Qwen3-Reranker-0.6B` - Model reranker cho document reranking

**Lưu ý về RAM:**
- Notebook này cần khoảng 8-12GB RAM
- Trên Colab Free (12.7GB RAM): Có thể chạy được nhưng cần restart runtime giữa 2 models
- Quantization chạy trên CPU, không tận dụng được GPU
- Nếu vẫn tràn RAM: Chạy từng model riêng biệt với restart runtime giữa mỗi model

## 1. Bootstrap - Thiết Lập Môi Trường

Cell này tự động phát hiện và cấu hình môi trường (local/colab/kaggle)

In [ ]:
# === BOOTSTRAP CELL - UNIVERSAL SETUP ===
import sys
import os
from pathlib import Path

# Cấu hình GitHub
GITHUB_USER = "n24q02m"
REPO_NAME = "n24q02m-kaggle-competitions"
BRANCH = "main"


# Phát hiện môi trường
def detect_env():
    if "google.colab" in sys.modules:
        return "colab"
    elif "kaggle_web_client" in sys.modules or os.path.exists("/kaggle"):
        return "kaggle"
    else:
        return "local"


ENV = detect_env()
print(f"Phát hiện môi trường: {ENV.upper()}")

# Thiết lập theo môi trường
if ENV == "local":
    # Local: Import trực tiếp từ repo
    repo_root = Path.cwd().parent
    if str(repo_root) not in sys.path:
        sys.path.insert(0, str(repo_root))

    from core import setup_env

    env = setup_env.setup()

else:
    # Cloud: Tải setup_env.py từ GitHub
    import requests
    import subprocess

    CORE_URL = (
        f"https://raw.githubusercontent.com/{GITHUB_USER}/{REPO_NAME}/{BRANCH}/core"
    )

    # Tải setup_env.py
    print("Đang tải setup_env.py...")
    response = requests.get(f"{CORE_URL}/setup_env.py")
    with open("setup_env.py", "w") as f:
        f.write(response.text)

    # Import và thiết lập
    import setup_env

    env = setup_env.setup(GITHUB_USER, REPO_NAME)

# Hiển thị thông tin môi trường
env.info()

## 2. Cấu Hình

Cấu hình các model và đường dẫn output

In [ ]:
import warnings

warnings.filterwarnings("ignore")


class CFG:
    # Model nguồn từ HuggingFace
    embedding_model_id = "Qwen/Qwen3-Embedding-0.6B"
    reranker_model_id = "Qwen/Qwen3-Reranker-0.6B"

    # HuggingFace username (thay đổi theo username của bạn)
    hf_username = "n24q02m"

    # Tên repo đích trên HuggingFace Hub
    embedding_repo_name = "Qwen3-Embedding-0.6B-ONNX"
    reranker_repo_name = "Qwen3-Reranker-0.6B-ONNX"

    # Thư mục output
    output_dir = Path.cwd() / "models"
    embedding_output_dir = output_dir / "qwen3-embedding-onnx"
    reranker_output_dir = output_dir / "qwen3-reranker-onnx"

    # Cài đặt ONNX Export
    opset_version = 14

    # Cài đặt Quantization
    quantize = True
    quantization_type = "dynamic"  # dynamic hoặc static


# Tạo thư mục output
CFG.output_dir.mkdir(parents=True, exist_ok=True)
CFG.embedding_output_dir.mkdir(parents=True, exist_ok=True)
CFG.reranker_output_dir.mkdir(parents=True, exist_ok=True)

print("Cấu hình:")
print(f"  - Embedding Model: {CFG.embedding_model_id}")
print(f"  - Reranker Model: {CFG.reranker_model_id}")
print(f"  - Thư mục Output: {CFG.output_dir}")
print(f"  - Quantization: {CFG.quantize} ({CFG.quantization_type})")
print(f"  - HuggingFace Username: {CFG.hf_username}")

## 3. Import Thư Viện

Import các thư viện cần thiết (đã được cài đặt từ requirements.txt hoặc sẵn có trên Colab/Kaggle)

In [ ]:
# Import các thư viện
import gc
import torch
import onnx
import onnxruntime as ort
import shutil
from transformers import AutoTokenizer
from optimum.onnxruntime import (
    ORTModelForFeatureExtraction,
    ORTModelForSequenceClassification,
)
from optimum.onnxruntime.configuration import AutoQuantizationConfig
from optimum.onnxruntime import ORTQuantizer


def clear_memory():
    """Giải phóng bộ nhớ RAM và GPU"""
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    print("Đã giải phóng bộ nhớ.")


print(f"PyTorch version: {torch.__version__}")
print(f"ONNX version: {onnx.__version__}")
print(f"ONNX Runtime version: {ort.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

## 4. Tải Xuống và Export Embedding Model sang ONNX

Tải xuống Qwen3-Embedding-0.6B từ HuggingFace và chuyển đổi sang định dạng ONNX

In [ ]:
print("=" * 60)
print("EMBEDDING MODEL - CHUYỂN ĐỔI ONNX")
print("=" * 60)

print(f"\nĐang tải model: {CFG.embedding_model_id}")
print("Quá trình này có thể mất vài phút...")

embedding_onnx_path = CFG.embedding_output_dir / "fp32"
embedding_onnx_path.mkdir(parents=True, exist_ok=True)

try:
    # Tải tokenizer
    embedding_tokenizer = AutoTokenizer.from_pretrained(
        CFG.embedding_model_id, trust_remote_code=True
    )

    # Export sang ONNX sử dụng optimum
    embedding_ort_model = ORTModelForFeatureExtraction.from_pretrained(
        CFG.embedding_model_id, export=True, trust_remote_code=True
    )

    # Lưu model và tokenizer
    embedding_ort_model.save_pretrained(embedding_onnx_path)
    embedding_tokenizer.save_pretrained(embedding_onnx_path)

    print(f"\nEmbedding model đã được export tại: {embedding_onnx_path}")
    print("Danh sách files:")
    for f in embedding_onnx_path.iterdir():
        size_mb = f.stat().st_size / (1024**2) if f.is_file() else 0
        print(f"  - {f.name} ({size_mb:.2f} MB)" if size_mb > 0 else f"  - {f.name}/")

    # Giải phóng bộ nhớ ngay sau khi lưu
    del embedding_ort_model
    del embedding_tokenizer
    clear_memory()

except Exception as e:
    print(f"Lỗi khi export embedding model: {e}")
    print("\nThử phương pháp thay thế với optimum-cli...")
    !optimum-cli export onnx --model {CFG.embedding_model_id} --task feature-extraction --trust-remote-code {embedding_onnx_path}
    clear_memory()

## 5. Quantize Embedding Model (INT8)

Áp dụng Dynamic Quantization để giảm kích thước model.

**Lưu ý:** Quantization trong ONNX Runtime chạy trên CPU, không tận dụng được GPU. Nếu Python API thất bại sẽ tự động fallback sang CLI.

In [ ]:
if CFG.quantize:
    print("=" * 60)
    print("EMBEDDING MODEL - QUANTIZATION (INT8)")
    print("=" * 60)

    embedding_fp32_path = CFG.embedding_output_dir / "fp32"
    embedding_int8_path = CFG.embedding_output_dir / "int8"
    embedding_int8_path.mkdir(parents=True, exist_ok=True)

    try:
        # Quantization trực tiếp trong Python
        quantizer = ORTQuantizer.from_pretrained(embedding_fp32_path)
        qconfig = AutoQuantizationConfig.avx512_vnni(is_static=False, per_channel=False)
        quantizer.quantize(save_dir=embedding_int8_path, quantization_config=qconfig)

        # Copy tokenizer files
        for f in embedding_fp32_path.iterdir():
            if not f.name.endswith(".onnx") and not f.name.endswith("_data"):
                if f.is_file():
                    shutil.copy(f, embedding_int8_path / f.name)

        print(f"\nModel đã quantize được lưu tại: {embedding_int8_path}")

        # So sánh kích thước
        fp32_size = sum(
            f.stat().st_size for f in embedding_fp32_path.glob("*.onnx")
        ) / (1024**2)
        int8_size = sum(
            f.stat().st_size for f in embedding_int8_path.glob("*.onnx")
        ) / (1024**2)

        print(f"\nSo sánh kích thước Model:")
        print(f"  FP32: {fp32_size:.2f} MB")
        print(f"  INT8: {int8_size:.2f} MB")
        print(f"  Giảm: {(1 - int8_size / fp32_size) * 100:.1f}%")

        del quantizer
        clear_memory()

    except Exception as e:
        print(f"Lỗi khi quantize embedding model: {e}")
        print("\nThử phương pháp CLI...")
        !optimum-cli onnxruntime quantize --onnx_model "{embedding_fp32_path}" --avx512_vnni -o "{embedding_int8_path}"

        # Copy tokenizer files
        for f in embedding_fp32_path.iterdir():
            if not f.name.endswith(".onnx") and not f.name.endswith("_data"):
                if f.is_file() and not (embedding_int8_path / f.name).exists():
                    shutil.copy(f, embedding_int8_path / f.name)
        clear_memory()
else:
    print("Quantization đã tắt. Bỏ qua...")

print("\n" + "=" * 60)
print("EMBEDDING MODEL - HOÀN THÀNH!")
print("=" * 60)

## 6. Tải Xuống và Export Reranker Model sang ONNX

Tải xuống Qwen3-Reranker-0.6B từ HuggingFace và chuyển đổi sang định dạng ONNX.

**Trên Colab:** Nếu vẫn bị tràn RAM, hãy restart runtime (Runtime -> Restart runtime) trước khi chạy cell này. Sau đó chạy lại từ Section 1-3, bỏ qua Section 4-5.

In [ ]:
print("=" * 60)
print("RERANKER MODEL - CHUYỂN ĐỔI ONNX")
print("=" * 60)

print(f"\nĐang tải model: {CFG.reranker_model_id}")
print("Quá trình này có thể mất vài phút...")

reranker_onnx_path = CFG.reranker_output_dir / "fp32"
reranker_onnx_path.mkdir(parents=True, exist_ok=True)

try:
    # Tải tokenizer
    reranker_tokenizer = AutoTokenizer.from_pretrained(
        CFG.reranker_model_id, trust_remote_code=True
    )

    # Export sang ONNX - Reranker là cross-encoder, sử dụng SequenceClassification
    reranker_ort_model = ORTModelForSequenceClassification.from_pretrained(
        CFG.reranker_model_id, export=True, trust_remote_code=True
    )

    # Lưu model và tokenizer
    reranker_ort_model.save_pretrained(reranker_onnx_path)
    reranker_tokenizer.save_pretrained(reranker_onnx_path)

    print(f"\nReranker model đã được export tại: {reranker_onnx_path}")
    print("Danh sách files:")
    for f in reranker_onnx_path.iterdir():
        size_mb = f.stat().st_size / (1024**2) if f.is_file() else 0
        print(f"  - {f.name} ({size_mb:.2f} MB)" if size_mb > 0 else f"  - {f.name}/")

    # Giải phóng bộ nhớ ngay sau khi lưu
    del reranker_ort_model
    del reranker_tokenizer
    clear_memory()

except Exception as e:
    print(f"Lỗi khi export reranker model: {e}")
    print("\nThử phương pháp thay thế với optimum-cli...")
    !optimum-cli export onnx --model {CFG.reranker_model_id} --task text-classification --trust-remote-code {reranker_onnx_path}
    clear_memory()

## 7. Quantize Reranker Model (INT8)

Áp dụng Dynamic Quantization để giảm kích thước model. Nếu Python API thất bại sẽ tự động fallback sang CLI.

In [ ]:
if CFG.quantize:
    print("=" * 60)
    print("RERANKER MODEL - QUANTIZATION (INT8)")
    print("=" * 60)

    reranker_fp32_path = CFG.reranker_output_dir / "fp32"
    reranker_int8_path = CFG.reranker_output_dir / "int8"
    reranker_int8_path.mkdir(parents=True, exist_ok=True)

    # Sử dụng subprocess để chạy quantization - giải phóng RAM sau khi xong
    quantize_script = f'''
import shutil
from pathlib import Path
from optimum.onnxruntime import ORTQuantizer
from optimum.onnxruntime.configuration import AutoQuantizationConfig

fp32_path = Path("{reranker_fp32_path}")
int8_path = Path("{reranker_int8_path}")

# Quantize
quantizer = ORTQuantizer.from_pretrained(fp32_path)
qconfig = AutoQuantizationConfig.avx512_vnni(is_static=False, per_channel=False)
quantizer.quantize(save_dir=int8_path, quantization_config=qconfig)

# Copy tokenizer files
for f in fp32_path.iterdir():
    if not f.name.endswith(".onnx") and not f.name.endswith("_data"):
        if f.is_file():
            shutil.copy(f, int8_path / f.name)

print("Quantization completed!")
'''

    try:
        result = subprocess.run(
            ["python", "-c", quantize_script],
            capture_output=True,
            text=True,
            timeout=600,  # 10 phút timeout
        )

        if result.returncode == 0:
            print(result.stdout)
            print(f"\nModel đã quantize được lưu tại: {reranker_int8_path}")

            # So sánh kích thước
            fp32_size = sum(
                f.stat().st_size for f in reranker_fp32_path.glob("*.onnx")
            ) / (1024**2)
            int8_size = sum(
                f.stat().st_size for f in reranker_int8_path.glob("*.onnx")
            ) / (1024**2)

            print(f"\nSo sánh kích thước Model:")
            print(f"  FP32: {fp32_size:.2f} MB")
            print(f"  INT8: {int8_size:.2f} MB")
            print(f"  Giảm: {(1 - int8_size / fp32_size) * 100:.1f}%")
        else:
            print(f"Lỗi: {result.stderr}")
            raise Exception(result.stderr)

    except Exception as e:
        print(f"Lỗi khi quantize reranker model: {e}")
        print("Bỏ qua quantization cho reranker model.")

    clear_memory()
else:
    print("Quantization đã tắt. Bỏ qua...")

print("\n" + "=" * 60)
print("RERANKER MODEL - HOÀN THÀNH!")
print("=" * 60)

## 8. Kiểm Tra Models ONNX

Kiểm tra các model ONNX đã export có hoạt động đúng không

In [ ]:
print("=" * 60)
print("KIỂM TRA MODEL")
print("=" * 60)

# Kiểm tra Embedding Model
print("\n--- Kiểm tra Embedding Model (FP32) ---")
try:
    embedding_fp32_path = CFG.embedding_output_dir / "fp32"

    # Tải model
    ort_embedding = ORTModelForFeatureExtraction.from_pretrained(embedding_fp32_path)
    tokenizer = AutoTokenizer.from_pretrained(embedding_fp32_path)

    # Kiểm tra inference
    test_text = "Đây là câu kiểm tra cho embedding model."
    inputs = tokenizer(test_text, return_tensors="pt", padding=True, truncation=True)
    outputs = ort_embedding(**inputs)

    print(f"Input: {test_text}")
    print(f"Output shape: {outputs.last_hidden_state.shape}")
    print("Embedding Model FP32: OK")

    # Giải phóng
    del ort_embedding, tokenizer, inputs, outputs
    clear_memory()
except Exception as e:
    print(f"Embedding Model FP32 Lỗi: {e}")

# Kiểm tra Embedding Model INT8
if CFG.quantize:
    print("\n--- Kiểm tra Embedding Model (INT8) ---")
    try:
        embedding_int8_path = CFG.embedding_output_dir / "int8"
        tokenizer = AutoTokenizer.from_pretrained(embedding_int8_path)
        ort_embedding_int8 = ORTModelForFeatureExtraction.from_pretrained(
            embedding_int8_path
        )

        test_text = "Đây là câu kiểm tra cho embedding model."
        inputs = tokenizer(
            test_text, return_tensors="pt", padding=True, truncation=True
        )
        outputs_int8 = ort_embedding_int8(**inputs)

        print(f"Output shape: {outputs_int8.last_hidden_state.shape}")
        print("Embedding Model INT8: OK")

        del ort_embedding_int8, tokenizer, inputs, outputs_int8
        clear_memory()
    except Exception as e:
        print(f"Embedding Model INT8 Lỗi: {e}")

In [ ]:
# Kiểm tra Reranker Model
print("\n--- Kiểm tra Reranker Model (FP32) ---")
try:
    reranker_fp32_path = CFG.reranker_output_dir / "fp32"

    # Tải model
    ort_reranker = ORTModelForSequenceClassification.from_pretrained(reranker_fp32_path)
    tokenizer_reranker = AutoTokenizer.from_pretrained(reranker_fp32_path)

    # Kiểm tra inference - Reranker nhận query và document
    query = "Machine learning là gì?"
    document = "Machine learning là một nhánh của trí tuệ nhân tạo cho phép hệ thống học từ dữ liệu."

    inputs_reranker = tokenizer_reranker(
        query, document, return_tensors="pt", padding=True, truncation=True
    )
    outputs_reranker = ort_reranker(**inputs_reranker)

    print(f"Query: {query}")
    print(f"Document: {document[:50]}...")
    print(f"Score: {outputs_reranker.logits}")
    print("Reranker Model FP32: OK")

    del ort_reranker, tokenizer_reranker, inputs_reranker, outputs_reranker
    clear_memory()
except Exception as e:
    print(f"Reranker Model FP32 Lỗi: {e}")

# Kiểm tra Reranker Model INT8
if CFG.quantize:
    print("\n--- Kiểm tra Reranker Model (INT8) ---")
    try:
        reranker_int8_path = CFG.reranker_output_dir / "int8"

        ort_reranker_int8 = ORTModelForSequenceClassification.from_pretrained(
            reranker_int8_path
        )
        tokenizer_reranker = AutoTokenizer.from_pretrained(reranker_int8_path)

        query = "Machine learning là gì?"
        document = "Machine learning là một nhánh của trí tuệ nhân tạo."
        inputs_reranker = tokenizer_reranker(
            query, document, return_tensors="pt", padding=True, truncation=True
        )
        outputs_reranker_int8 = ort_reranker_int8(**inputs_reranker)

        print(f"Score: {outputs_reranker_int8.logits}")
        print("Reranker Model INT8: OK")

        del (
            ort_reranker_int8,
            tokenizer_reranker,
            inputs_reranker,
            outputs_reranker_int8,
        )
        clear_memory()
    except Exception as e:
        print(f"Reranker Model INT8 Lỗi: {e}")

## 9. Tạo Model Card

Tạo file README.md (Model Card) cho mỗi model để upload lên HuggingFace Hub

In [ ]:
def create_model_card(model_name, source_model, model_type, hf_username):
    """Tạo nội dung Model Card cho HuggingFace Hub"""

    task_name = (
        "feature-extraction"
        if model_type == "embedding"
        else "text-classification (reranking)"
    )
    ort_class = (
        "FeatureExtraction" if model_type == "embedding" else "SequenceClassification"
    )

    return f"""---
license: apache-2.0
language:
  - en
  - zh
tags:
  - onnx
  - optimum
  - {model_type}
  - qwen3
  - sentence-similarity
base_model: {source_model}
library_name: optimum
pipeline_tag: {"feature-extraction" if model_type == "embedding" else "text-classification"}
---

# {model_name}

Phiên bản ONNX của [{source_model}](https://huggingface.co/{source_model})

## Thông Tin Model

| Thuộc tính | Giá trị |
|------------|---------|
| Model nguồn | [{source_model}](https://huggingface.co/{source_model}) |
| Định dạng | ONNX |
| Quantization | FP32 + INT8 Dynamic |
| Task | {task_name} |

## Các Phiên Bản

- `fp32/`: Model full precision (FP32)
- `int8/`: Model đã quantize INT8 (dynamic quantization)

## Cách Sử Dụng

### Với Optimum

```python
from optimum.onnxruntime import ORTModelFor{ort_class}
from transformers import AutoTokenizer

# Tải model FP32
model = ORTModelFor{ort_class}.from_pretrained(
    "{hf_username}/{model_name}",
    subfolder="fp32"
)
tokenizer = AutoTokenizer.from_pretrained(
    "{hf_username}/{model_name}",
    subfolder="fp32"
)

# Hoặc tải model INT8 để inference nhanh hơn
model_int8 = ORTModelFor{ort_class}.from_pretrained(
    "{hf_username}/{model_name}",
    subfolder="int8"
)
```

### Với ONNX Runtime trực tiếp

```python
import onnxruntime as ort
from transformers import AutoTokenizer
import numpy as np

# Tải tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    "{hf_username}/{model_name}",
    subfolder="fp32"
)

# Tạo ONNX Runtime session
session = ort.InferenceSession("fp32/model.onnx")

# Inference
text = "Your text here"
inputs = tokenizer(text, return_tensors="np")
outputs = session.run(None, dict(inputs))
```

## License

Model này được phát hành theo giấy phép Apache 2.0, tuân theo giấy phép của model gốc.

## Ghi Công

- Model gốc: [Qwen Team](https://huggingface.co/Qwen)
- Chuyển đổi ONNX: Sử dụng [Optimum](https://huggingface.co/docs/optimum)
"""

In [ ]:
print("=" * 60)
print("TẠO MODEL CARD")
print("=" * 60)

# Tạo Model Card cho Embedding Model
print("\n--- Tạo Model Card cho Embedding Model ---")
embedding_readme = create_model_card(
    CFG.embedding_repo_name, CFG.embedding_model_id, "embedding", CFG.hf_username
)
embedding_readme_path = CFG.embedding_output_dir / "README.md"
embedding_readme_path.write_text(embedding_readme, encoding="utf-8")
print(f"Đã tạo: {embedding_readme_path}")

# Tạo Model Card cho Reranker Model
print("\n--- Tạo Model Card cho Reranker Model ---")
reranker_readme = create_model_card(
    CFG.reranker_repo_name, CFG.reranker_model_id, "reranker", CFG.hf_username
)
reranker_readme_path = CFG.reranker_output_dir / "README.md"
reranker_readme_path.write_text(reranker_readme, encoding="utf-8")
print(f"Đã tạo: {reranker_readme_path}")

print("\nModel Cards đã được tạo thành công!")

## 10. Hướng Dẫn Upload Thủ Công Lên HuggingFace Hub

Hướng dẫn từng bước để upload models lên HuggingFace Hub

In [ ]:
print("=" * 60)
print("HƯỚNG DẪN UPLOAD THỦ CÔNG")
print("=" * 60)

print(
    """
Có 2 cách để upload models lên HuggingFace Hub:

=== CÁCH 1: Sử dụng HuggingFace CLI ===

1. Đăng nhập HuggingFace (chạy trong terminal):
   $ huggingface-cli login

2. Tạo repository trên HuggingFace:
   $ huggingface-cli repo create {embedding_repo} --type model
   $ huggingface-cli repo create {reranker_repo} --type model

3. Upload Embedding Model:
   $ huggingface-cli upload {hf_user}/{embedding_repo} {embedding_dir} .

4. Upload Reranker Model:
   $ huggingface-cli upload {hf_user}/{reranker_repo} {reranker_dir} .

=== CÁCH 2: Sử dụng Web UI ===

1. Truy cập: https://huggingface.co/new
2. Tạo model repository mới
3. Click "Add file" -> "Upload files"
4. Upload toàn bộ thư mục model

=== CẤU TRÚC THƯ MỤC CẦN UPLOAD ===
""".format(
        hf_user=CFG.hf_username,
        embedding_repo=CFG.embedding_repo_name,
        reranker_repo=CFG.reranker_repo_name,
        embedding_dir=CFG.embedding_output_dir,
        reranker_dir=CFG.reranker_output_dir,
    )
)

# Hiển thị cấu trúc thư mục
print(f"\n{CFG.embedding_repo_name}/")
for item in sorted(CFG.embedding_output_dir.rglob("*")):
    rel_path = item.relative_to(CFG.embedding_output_dir)
    indent = "  " * len(rel_path.parts)
    if item.is_file():
        size_mb = item.stat().st_size / (1024**2)
        print(f"{indent}{item.name} ({size_mb:.2f} MB)")
    else:
        print(f"{indent}{item.name}/")

print(f"\n{CFG.reranker_repo_name}/")
for item in sorted(CFG.reranker_output_dir.rglob("*")):
    rel_path = item.relative_to(CFG.reranker_output_dir)
    indent = "  " * len(rel_path.parts)
    if item.is_file():
        size_mb = item.stat().st_size / (1024**2)
        print(f"{indent}{item.name} ({size_mb:.2f} MB)")
    else:
        print(f"{indent}{item.name}/")

In [ ]:
# In ra các lệnh CLI cụ thể để copy-paste
print("=" * 60)
print("LỆNH CLI ĐỂ UPLOAD (copy và chạy trong terminal)")
print("=" * 60)

commands = f"""
# 1. Đăng nhập HuggingFace
huggingface-cli login

# 2. Tạo repositories
huggingface-cli repo create {CFG.embedding_repo_name} --type model
huggingface-cli repo create {CFG.reranker_repo_name} --type model

# 3. Upload Embedding Model
huggingface-cli upload {CFG.hf_username}/{CFG.embedding_repo_name} "{CFG.embedding_output_dir}" .

# 4. Upload Reranker Model
huggingface-cli upload {CFG.hf_username}/{CFG.reranker_repo_name} "{CFG.reranker_output_dir}" .
"""

print(commands)

print("\n" + "=" * 60)
print("SAU KHI UPLOAD, MODELS SẼ CÓ TẠI:")
print("=" * 60)
print(
    f"  - Embedding: https://huggingface.co/{CFG.hf_username}/{CFG.embedding_repo_name}"
)
print(
    f"  - Reranker: https://huggingface.co/{CFG.hf_username}/{CFG.reranker_repo_name}"
)

## 11. Tổng Kết

Tổng kết quá trình chuyển đổi và các bước tiếp theo

In [ ]:
print("\n" + "=" * 60)
print("TỔNG KẾT CHUYỂN ĐỔI")
print("=" * 60)

print("\nModels nguồn:")
print(f"  - Embedding: {CFG.embedding_model_id}")
print(f"  - Reranker: {CFG.reranker_model_id}")

print("\nModels đã chuyển đổi:")
print(f"  - Embedding ONNX: {CFG.embedding_output_dir}")
print(f"  - Reranker ONNX: {CFG.reranker_output_dir}")


# Tính tổng kích thước
def get_folder_size(folder):
    return sum(f.stat().st_size for f in Path(folder).rglob("*") if f.is_file()) / (
        1024**2
    )


print("\nKích thước:")
try:
    emb_size = get_folder_size(CFG.embedding_output_dir)
    rer_size = get_folder_size(CFG.reranker_output_dir)
    print(f"  - Embedding: {emb_size:.2f} MB")
    print(f"  - Reranker: {rer_size:.2f} MB")
    print(f"  - Tổng cộng: {emb_size + rer_size:.2f} MB")
except:
    print("  - Không thể tính kích thước")

print("\nCác phiên bản có sẵn:")
print("  - FP32 (Full Precision)")
if CFG.quantize:
    print("  - INT8 (Dynamic Quantization)")

print("\nBước tiếp theo:")
print("  1. Chạy các lệnh CLI ở Section 10 để upload lên HuggingFace Hub")
print("  2. Kiểm tra repositories trên HuggingFace")
print("  3. Test models từ HuggingFace Hub")

print("\n" + "=" * 60)
print("HOÀN THÀNH!")
print("=" * 60)